we are trying to demonstarte simple LSTM based RNN implementaton here

we have a sentence ""i love neural networks"

next we will design a model, such that if ip is "i love neu" 
op will be "i love neur"

for demonstration we will only train on "i love neural networks"

I’ve overfit the model by training on multiple iterations using the same sequence again and again. I’d like to highlight again that , this post is not aimed at teaching RNN or LSTM. It’s just a walk through of working implementation of character generation model LSTM RNN using pytorch python library

while training :
for LSTM model, 
we will give a sequence at a time and it will predict a next letter

for example  torch.Size([22, 1, 28])

above is shape of input, 22 is squence length and 28 vector length for each character.

if input is :'i love neur'
then  results of model are:

out=predict('i love neur') #input shape is torch.Size([11, 1, 28])

print(out)# a

here ([11,1,28]), 11 is sequence length, 1 is batch size and 28 is vector representation of every char in sequence.



In [84]:
import torch
import torch.nn as nn

In [85]:
data="i love neural networks"

seq_len=len(data)
print(seq_len)

22


In [86]:
import string
letters = string.ascii_lowercase+' #'
n_letters = len(letters)
print('Letter set is '+letters)

Letter set is abcdefghijklmnopqrstuvwxyz #


In [87]:
#my neural network

class LSTMNN(nn.Module):
    def __init__(self,input_dim,hidden_dim):
       super(LSTMNN,self).__init__()
       self.input_dim=input_dim
       self.hidden_dim=hidden_dim
       ##LSTM takes, input dimensions, hidden dimensions and num layers in case of stacked LSTMs (Default is 1)
       self.LSTM=nn.LSTM(input_dim,hidden_dim)


#Input must be 3 dimensional (Sequence len, batch, input dimensions) 
#hc is a tuple which contains the vectors h (hidden/feedback) and c (cell state vector  )

    def forward(self,inp,hc):
     ##this gives outut for each input and also (hidden and cell state vector)
     output,_=self.LSTM(inp,hc)
     return output



In [88]:
#Dimensions of output of neural network is (seq_len, batch , hidden_dim). Since we want output dimensions to be
#the same as n_letters, hidden_dim = n_letters (output dimensions = hidden_dimensions)
hidden_dim = n_letters

#Invoking model. Input dimensions = n_letters i.e 28. output dimensions = hidden_dimensions = 28

model = LSTMNN(n_letters,hidden_dim)

#I'm using Adam optimizer here
optimizer = torch.optim.Adam(params = model.parameters(),lr=0.01)

#Loss function is CrossEntropyLoss
LOSS = torch.nn.CrossEntropyLoss()

The model we are trying to build is a character generation model. That means, we will input the sequence , “i love neural networks” to the model. Since this is a supervised technique, we must have a output to train the model. I’m taking the output for a sequence is ,next letter in the sequence. That means, for the input sequence “i love neu”, output character must be “r”, since r is the next letter in the original sequence. Similarily, for input sequence “i love neura”, output sequence is “l”. Now. let’s try to build actual outputs. Let’s call them ‘targets’.

In [89]:
print(data)
for x in data[1:]+'#':
  print(x)

i love neural networks
 
l
o
v
e
 
n
e
u
r
a
l
 
n
e
t
w
o
r
k
s
#


In [91]:
targets=[]
print("letters is :",letters)
print("data is :",data)

for x in data[1:]+'#':
  targets.append(letters.find(x))

print("targets is :",targets)
print("\nconverting it into tensors")
targets=torch.tensor(targets)
print(targets)

letters is : abcdefghijklmnopqrstuvwxyz #
data is : i love neural networks
targets is : [26, 11, 14, 21, 4, 26, 13, 4, 20, 17, 0, 11, 26, 13, 4, 19, 22, 14, 17, 10, 18, 27]

converting it into tensors
tensor([26, 11, 14, 21,  4, 26, 13,  4, 20, 17,  0, 11, 26, 13,  4, 19, 22, 14,
        17, 10, 18, 27])


This encoding is quite simple and straight forward. There are 28 allowed letters. (26 letters + 1 space + 1 ‘#’). 

I convert each of my character into a 28 dimensional vector. All the other dimensions have value 0, except the index of character in our string letters, which we defined above

In [92]:
print(n_letters)
print(letters)

def ltt(ch):
  ans=torch.zeros(n_letters)
  ans[letters.find(ch)]=1
  return ans

print("\n\n\n")
print("Encoding of 'b' ",ltt('b'))
print("Encoding of '#' ",ltt('#'))  

28
abcdefghijklmnopqrstuvwxyz #




Encoding of 'b'  tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Encoding of '#'  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])


In [93]:
#Now, let’s build input sequence to train
print(data)
inpl=[]

for c in data:
  inpl.append(ltt(c))

print(inpl)

#Concatenates the given sequence of seq tensors in the given dimension. All tensors must either have the same shape (except in the concatenating dimension) or be empty.
inp = torch.cat(inpl,dim=0)

print(inp)

#Reshape tensor into 3 dimensions (sequence length, batches = 1, dimensions = n_letters (28))
inp=inp.view(seq_len,1,n_letters)

print("shape  of inp",inp.shape) # shape of inp torch.Size([22, 1, 28])
#22 here becuase "i love neural networks" is of 22 length
#1 becuase we are only passing one sentence in a batch
#28 becuase , while 22 char's will be fed to network at 22 time steps ('i',' ','l','o','v','e' .......)all at once and each is reperesented by vector of length 28

i love neural networks
[tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.

In [95]:
print(inp)
#1st tensor below is for i
#[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
#          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

TRAINING


In [96]:
import time
start=time.time()
n_iters=150

for itr in range(n_iters):
  model.zero_grad()
  optimizer.zero_grad()

  #initialse h and c
  h=torch.rand(hidden_dim).view(1,1,hidden_dim)
  c=torch.rand(hidden_dim).view(1,1,hidden_dim)
  
  #find output
  output=model(inp,(h,c))

  #Reshape the output to 2 dimensions. This is done, so that we can compare with target and get loss
  output=output.view(seq_len,n_letters)
  
  #find loss
  loss=LOSS(output,targets)
  #backpropogate
  loss.backward()
  #optimise
  optimizer.step

print((time.time()-start))

0.5509171485900879


In [97]:
print(output.shape)
print(targets.shape)
print(loss)
print(targets)


#    l  o v  e     n e u   r a  l    n  e  t w  o  r  k  s
#26,11,14,21,4,26,13,4,20,17,0,11,26,13,4,19,22,14,17,10,18,27
print(output)

torch.Size([22, 28])
torch.Size([22])
tensor(3.3675, grad_fn=<NllLossBackward>)
tensor([26, 11, 14, 21,  4, 26, 13,  4, 20, 17,  0, 11, 26, 13,  4, 19, 22, 14,
        17, 10, 18, 27])
tensor([[ 1.2021e-01,  1.9891e-01,  3.5588e-02,  1.0200e-02,  2.0669e-01,
         -4.1973e-03,  6.9070e-02,  1.2849e-01,  1.5843e-01, -5.4210e-02,
          2.3235e-01,  1.6107e-01, -3.0223e-03,  9.5466e-02,  1.7399e-01,
         -1.2821e-02,  1.2816e-01,  6.1702e-02,  9.8040e-02,  8.0471e-02,
          3.5173e-01,  2.7327e-01,  1.1794e-01,  4.1162e-01, -8.3781e-02,
          9.2949e-02, -1.3487e-01,  2.1626e-01],
        [ 1.1792e-01,  6.4241e-02,  3.9284e-02, -5.3765e-02,  8.5921e-02,
          2.9360e-02,  1.1810e-01,  5.0615e-02,  1.6627e-01, -1.2917e-01,
          7.5162e-02,  3.0393e-02,  2.6984e-02,  3.8626e-02,  2.8889e-02,
         -4.0855e-02,  4.3367e-02,  6.3578e-02,  4.2251e-02, -4.2544e-02,
          2.1858e-01,  9.9565e-02, -2.3334e-02,  2.8247e-01, -4.4822e-02,
          3.3417e-03, -1.3

In [98]:
#Utility function that predicts next letter , given the sequence

def getLine(s):
    ans = []
    for c in s:
        ans.append(ltt(c))
    return torch.cat(ans,dim=0).view(len(s),1,n_letters)
    

def predict(s):
  inp=getLine(s)

  h=torch.rand(1,1,hidden_dim)
  c=torch.rand(1,1,hidden_dim)

  out=model(inp,(h,c))

  return letters[out[-1][0].topk(1)[1].detach().numpy().item()]##torch.topk ... Returns the k largest elements of the given input tensor along a given dimension k==1


**combining everything**

In [1]:
import torch
import torch.nn as nn
import string

data = "i love neural networks"
EOF = "#"
#data = data+EOF
data = data.lower()

seq_len = len(data)

letters = string.ascii_lowercase+' #'
print('Letter set is '+letters)
n_letters = len(letters)
print(letters)

#letter to tensor
def ltt(ch):
    ans = torch.zeros(n_letters)
    ans[letters.find(ch)]=1
    return ans
    
def getLine(s):
    ans = []
    for c in s:
        ans.append(ltt(c))
    return torch.cat(ans,dim=0).view(len(s),1,n_letters)
    
class MyLSTM(nn.Module):
    def __init__(self,input_dim,hidden_dim):
        super(MyLSTM,self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        #LSTM takes, input_dim, hidden_dim and num_layers incase of stacked LSTMs
        self.LSTM = nn.LSTM(input_dim,hidden_dim)
        self.LNN = nn.Linear(hidden_dim,input_dim)
        
    #Input must be 3 dimensional (seq_len, batch, input_dim). 
    #hc is a tuple of hidden and cell state vector. Each of them have shape (1,1,hidden_dim)
    def forward(self,inp,hc):
        #this gives outut for each input in the sequence and also (hidden and cell state vector)
        #Dimensions of output vector is (seq_len,batch,hidden_dim)
        output,_= self.LSTM(inp,hc)
        return self.LNN(output)
        #return output
        

#Dimensions of output of neural network is (seq_len, batch , hidden_dim). Since we want output dimensions to be
#the same as n_letters, hidden_dim = n_letters (**output dimensions = hidden_dimensions)
hidden_dim = n_letters     
#Invoking model. Input dimensions = n_letters i.e 28. output dimensions = hidden_dimensions = 28
model = MyLSTM(n_letters,hidden_dim)
#I'm using Adam optimizer here
optimizer = torch.optim.Adam(params = model.parameters(),lr=0.01)
#Loss function is CrossEntropyLoss
LOSS = torch.nn.CrossEntropyLoss()

#List to store targets
targets = []
#Iterate through all chars in the sequence, starting from second letter. Since output for 1st letter is the 2nd letter
for x in data[1:]+'#':
    #Find the target index. For a, it is 0, For 'b' it is 1 etc..
    targets.append(letters.find(x))
#Convert into tensor
targets = torch.tensor(targets)
    
n_iters = 400

#List to store input
inpl = []
#Iterate through all inputs in the sequence
for c in data:
    #Convert into tensor
    inpl.append(ltt(c))
#Convert list to tensor
inp = torch.cat(inpl,dim=0)
#Reshape tensor into 3 dimensions (sequence length, batches = 1, dimensions = n_letters (28))
inp = inp.view(seq_len,1,n_letters)


#Let's note down start time to track the training time
import time
start = time.time()
#Number of iterations
n_iters = 150
for itr in range(n_iters):
    #Zero the previosus gradients
    model.zero_grad()
    optimizer.zero_grad()
    #Initialize h and c vectors
    h = torch.rand(hidden_dim).view(1,1,hidden_dim)
    c = torch.rand(hidden_dim).view(1,1,hidden_dim)
    #Find the output
    output = model(inp,(h,c))
    #Reshape the output to 2 dimensions. This is done, so that we can compare with target and get loss
    output = output.view(seq_len,n_letters)
    #Find loss
    loss = LOSS(output,targets)
    #Print loss for every 10th iteration
    if itr%10==0:
        print('Iteration : '+str(itr)+' Loss : '+str(loss) )
    #Back propagate the loss
    loss.backward()
    #Perform weight updation
    optimizer.step()
    
print('Time taken to train : '+str(time.time()-start)+" seconds")
 

#This utility method predicts the next letter given the sequence   
def predict(s):
    #Get the vector for input
    inp = getLine(s)
    #Initialize h and c vectors
    h = torch.rand(1,1,hidden_dim)
    c = torch.rand(1,1,hidden_dim)
    #Get the output
    print("input shape is {}".format(inp.shape))
    out = model(inp,(h,c))
    #Find the corresponding letter from the output
    return letters[out[-1][0].topk(1)[1].detach().numpy().item()]
         

Letter set is abcdefghijklmnopqrstuvwxyz #
abcdefghijklmnopqrstuvwxyz #
Iteration : 0 Loss : tensor(3.3682, grad_fn=<NllLossBackward>)
Iteration : 10 Loss : tensor(2.5280, grad_fn=<NllLossBackward>)
Iteration : 20 Loss : tensor(2.2620, grad_fn=<NllLossBackward>)
Iteration : 30 Loss : tensor(1.8049, grad_fn=<NllLossBackward>)
Iteration : 40 Loss : tensor(1.2759, grad_fn=<NllLossBackward>)
Iteration : 50 Loss : tensor(0.7552, grad_fn=<NllLossBackward>)
Iteration : 60 Loss : tensor(0.3560, grad_fn=<NllLossBackward>)
Iteration : 70 Loss : tensor(0.1730, grad_fn=<NllLossBackward>)
Iteration : 80 Loss : tensor(0.0861, grad_fn=<NllLossBackward>)
Iteration : 90 Loss : tensor(0.0537, grad_fn=<NllLossBackward>)
Iteration : 100 Loss : tensor(0.0385, grad_fn=<NllLossBackward>)
Iteration : 110 Loss : tensor(0.0283, grad_fn=<NllLossBackward>)
Iteration : 120 Loss : tensor(0.0224, grad_fn=<NllLossBackward>)
Iteration : 130 Loss : tensor(0.0194, grad_fn=<NllLossBackward>)
Iteration : 140 Loss : tensor

In [2]:
out=predict('i love ne')
print(out)

input shape is torch.Size([9, 1, 28])
u


In [3]:
out=predict('i love neur')
print(out)

input shape is torch.Size([11, 1, 28])
a


In [117]:
out=predict('i love neural ne')
print(out)

t


In [118]:
out=predict('i lov')
print(out)

e
